Countries Analysis

In [14]:
import pandas as pd
import requests
import json
import awswrangler as wr

In [15]:
# Fecthing the dat from api

api_url = 'https://restcountries.com/v3.1/all'
response = requests.get(api_url)
data = response.json()

In [16]:
# normalizing the JSON data 

df = pd.json_normalize(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Columns: 897 entries, tld to languages.nau
dtypes: bool(3), float64(20), int64(1), object(873)
memory usage: 1.7+ MB


In [17]:
# Dropping the unecessary columns 

# Multi column dropping
df_cols_to_drop = [col for col in df.columns if 'demonyms' in col or 'translations' in col or 'currencies' in col or 'coatOfArms' in col or 'postalCode' in col or 'name.nativeName' in col or 'name.tsn' in col or 'idd' in col or 'altSpellings' in col or 'gini' in col  or 'languages' in col]
df.drop(columns=df_cols_to_drop, inplace=True)

# Single column dropping 
df.drop(columns='flag', inplace = True)


In [18]:
# No concatenating of outputs
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_item', None)

In [19]:
# Single column cleaning

df['continents'] = df["continents"].astype(str).str.strip('[]\'')
df['capital'] = df["capital"].apply(str).str.strip('[]\'\"')

# Apply to the whole of dataframe
for col in df.columns:
    df[col] = df[col].apply(lambda col: str(col).strip('[]\'\"'))

df['borders'] = df['borders'].astype(str).str.replace("'","",regex= False)

df.head(1)

,tld,cca2,ccn3,cioc,independent,status,unMember,capital,region,subregion,latlng,landlocked,borders,area,cca3,population,fifa,timezones,continents,startOfWeek,name.common,name.official,maps.googleMaps,maps.openStreetMaps,car.signs,car.side,flags.png,flags.svg,flags.alt,capitalInfo.latlng
0,.er,ER,232,ERI,True,officially-assigned,True,Asmara,Africa,Eastern Africa,"15.0, 39.0",False,"DJI, ETH, SDN",117600.0,ERI,5352000,ERI,UTC+03:00,Africa,monday,Eritrea,State of Eritrea,https://goo.gl/maps/HRyqUpnPwwG6jY5j6,https://www.openstreetmap.org/relation/296961,ER,right,https://flagcdn.com/w320/er.png,https://flagcdn.com/er.svg,"The flag of Eritrea comprises three triangles — a large red isosceles triangle with its base spanning the hoist end and its apex at the midpoint on the fly end, and a green and blue right-angled triangle above and beneath the red triangle. On the hoist side of the red triangle is a golden vertical olive branch encircled by a golden olive wreath.","15.33, 38.93"


In [20]:
# Splitting of column in seperate for expansion of columns

df[['Latitude', 'Longitude']] = df['latlng'].str.split(',', n = 1, expand = True)
df[['Capital latitude', 'Capital Longitude']] = df['capitalInfo.latlng'].str.split(',', n = 1, expand = True)


columnsDropped = ['latlng', 'capitalInfo.latlng']
df.drop(columns = 'latlng', inplace = True)
df.head(1)

,tld,cca2,ccn3,cioc,independent,status,unMember,capital,region,subregion,landlocked,borders,area,cca3,population,fifa,timezones,continents,startOfWeek,name.common,name.official,maps.googleMaps,maps.openStreetMaps,car.signs,car.side,flags.png,flags.svg,flags.alt,capitalInfo.latlng,Latitude,Longitude,Capital latitude,Capital Longitude
0,.er,ER,232,ERI,True,officially-assigned,True,Asmara,Africa,Eastern Africa,False,"DJI, ETH, SDN",117600.0,ERI,5352000,ERI,UTC+03:00,Africa,monday,Eritrea,State of Eritrea,https://goo.gl/maps/HRyqUpnPwwG6jY5j6,https://www.openstreetmap.org/relation/296961,ER,right,https://flagcdn.com/w320/er.png,https://flagcdn.com/er.svg,"The flag of Eritrea comprises three triangles — a large red isosceles triangle with its base spanning the hoist end and its apex at the midpoint on the fly end, and a green and blue right-angled triangle above and beneath the red triangle. On the hoist side of the red triangle is a golden vertical olive branch encircled by a golden olive wreath.","15.33, 38.93",15.0,39.0,15.33,38.93


In [21]:
# Capital first letter in startOfWeek Column

def weekConversion(days):
    weekday = ""
    for i in range(0,len(days)):
        if i == 0:
            i = str(days[i]).upper()
            weekday += i
        else:
            weekday += days[i]
    return weekday
    
df['startOfWeek'] = df['startOfWeek'].apply(weekConversion)

In [22]:
df['status'].value_counts()

officially-assigned    249
user-assigned            1
Name: status, dtype: int64

In [23]:
df_cleaned_data = df.copy()

In [ ]:
wr.s3.to_parquet(df_cleaned_data,path="s3_file_path",index=False)

{'paths': ['s3://countrystore1/projdata/cleanedData.parquet'],
 'partitions_values': {}}

In [ ]:

# Read from S3
df_New = wr.s3.read_parquet("s3_file_Path")
print(df_New.head())


   tld cca2 ccn3 cioc independent               status unMember    capital  \
0  .er   ER  232  ERI        True  officially-assigned     True     Asmara   
1  .cm   CM  120  CMR        True  officially-assigned     True    Yaoundé   
2  .me   ME  499  MNE        True  officially-assigned     True  Podgorica   
3  .fj   FJ  242  FIJ        True  officially-assigned     True       Suva   
4  .tn   TN  788  TUN        True  officially-assigned     True      Tunis   

    region         subregion landlocked                       borders  \
0   Africa    Eastern Africa      False                 DJI, ETH, SDN   
1   Africa     Middle Africa      False  CAF, TCD, COG, GNQ, GAB, NGA   
2   Europe  Southeast Europe      False       ALB, BIH, HRV, UNK, SRB   
3  Oceania         Melanesia      False                           nan   
4   Africa   Northern Africa      False                      DZA, LBY   

       area cca3 population fifa  timezones continents startOfWeek  \
0  117600.0  ERI    53